Categorizing a dataset and applying a function to each group, whether an aggregation
or transformation, is often a critical component of a data analysis workflow. After
loading, merging, and preparing a dataset, you may need to compute group statistics
or possibly pivot tables for reporting or visualization purposes. pandas provides a
flexible groupby interface, enabling you to slice, dice, and summarize datasets in a natural way.


- Split a pandas object into pieces using one or more keys (in the form of functions,
arrays, or DataFrame column names)
- Calculate group summary statistics, like count, mean, or standard deviation, or a
user-defined function
- Apply within-group transformations or other manipulations, like normalization,
linear regression, rank, or subset selection
- Compute pivot tables and cross-tabulations
- Perform quantile analysis and other statistical group analyses

#<font color='blue'>1. GroupBy Mechanics</font>

Hadley Wickham, an author of many popular packages for the R programming language,
coined the term split-apply-combine for describing group operations. In the
first stage of the process, data contained in a pandas object, whether a Series, Data‐
Frame, or otherwise, is split into groups based on one or more keys that you provide.
The splitting is performed on a particular axis of an object. For example, a DataFrame
can be grouped on its rows (axis=0) or its columns (axis=1). Once this is done, a
function is applied to each group, producing a new value. Finally, the results of all
those function applications are combined into a result object.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,0.332396,-0.394612
1,a,two,-0.011710,1.617691
2,b,one,0.171690,-0.230081
3,b,two,1.129776,0.341716
4,a,one,-0.680759,0.010341


Suppose you wanted to compute the mean of the data1 column using the labels from key1.

In [ ]:
groupdf = df['data1'].groupby(df['key1'])

groupdf

In [ ]:
groupdf.mean()

key1
a   -0.120024
b    0.650733
Name: data1, dtype: float64

The important thing here is that the data(a series) has been aggregated according to the group key, producing a new series that is now indexed by the unique values in the `key1` column.

if instead we had passes multiple arrays as a list, we;d get something different

In [ ]:
groupdf = df['data1'].groupby([df['key1'],df['key2']]).mean()

In [ ]:
groupdf

key1  key2
a     one    -0.174181
      two    -0.011710
b     one     0.171690
      two     1.129776
Name: data1, dtype: float64

In [ ]:
groupdf.unstack()

key2,one,two
key1,,
a,-0.174181,-0.011710
b,0.171690,1.129776


In this example, the group keys are all series, though they could be any arrays of the right length.

In [ ]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])

df['data1'].groupby([states,years]).mean().unstack()

,2005,2006
California,-0.011710,0.171690
Ohio,0.731086,-0.680759


Frequently, the grouping information is found in the same DataFrame as the data you want to work on. 

In [ ]:
df.groupby(df['key1']).mean()

,data1,data2
key1,,
a,-0.120024,0.411140
b,0.650733,0.055817


### <font color='blue'>**Iterating Over Groups**</font>

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data. Consider the following 

In [ ]:
for name,group in df.groupby('key1'):
  print(name)
  print(group)

a
  key1 key2     data1     data2
0    a  one  0.332396 -0.394612
1    a  two -0.011710  1.617691
4    a  one -0.680759  0.010341
b
  key1 key2     data1     data2
2    b  one  0.171690 -0.230081
3    b  two  1.129776  0.341716


In the case of multiple keys, the frist element in the tuple will be a tuple of key values

In [ ]:
for (k1,k2), group in df.groupby(['key1','key2']):
  print((k1,k2))
  print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.332396 -0.394612
4    a  one -0.680759  0.010341
('a', 'two')
  key1 key2    data1     data2
1    a  two -0.01171  1.617691
('b', 'one')
  key1 key2    data1     data2
2    b  one  0.17169 -0.230081
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.129776  0.341716


### <font color='blue'>**Selecting a Column or Subset of COlumns**</font>

In [ ]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns = ['a','b','c','d','e'],
                      index = ['joe','Steve','Wes','Jim','Travis'])

people.iloc[2:3,[1,2]] = np.nan

In [ ]:
people

,a,b,c,d,e
joe,0.107586,-0.301705,-1.276279,-1.229901,2.795426
Steve,0.777327,0.411952,0.236881,0.071740,1.338883
Wes,0.029689,NaN,NaN,0.252610,0.016942
Jim,-0.167844,-0.631398,-0.091508,-0.939443,0.455956
Travis,0.618666,-1.227650,1.530363,1.550796,0.041313


In [ ]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.030570,-0.933103,-1.367787,-1.916734,3.268324
5,0.777327,0.411952,0.236881,0.071740,1.338883
6,0.618666,-1.227650,1.530363,1.550796,0.041313


In [ ]:
key_lst = ['one','one','one','two','two']

In [ ]:
people.groupby([len,key_lst]).sum()

a         b         c         d         e
3 one  0.137275 -0.301705 -1.276279 -0.977291  2.812368
  two -0.167844 -0.631398 -0.091508 -0.939443  0.455956
5 one  0.777327  0.411952  0.236881  0.071740  1.338883
6 two  0.618666 -1.227650  1.530363  1.550796  0.041313

#<font color='blue'>2. Data Aggregation</font>

Aggregations refer to any data transformation that produces scalar values from arrays

In [ ]:
df

,key1,key2,data1,data2
0,a,one,0.332396,-0.394612
1,a,two,-0.011710,1.617691
2,b,one,0.171690,-0.230081
3,b,two,1.129776,0.341716
4,a,one,-0.680759,0.010341


In [ ]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.263575
b    1.033967
Name: data1, dtype: float64

In [ ]:
grouped

To use your own aggregation functions, pass any function that aggregates an array to the aggregate or agg method.

In [ ]:
def peak_to_peak(arr):
  return arr.max() - arr.min()

In [ ]:
grouped.agg(peak_to_peak)

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,data1,data2
key1,,
a,1.013156,2.012303
b,0.958086,0.571798


In [ ]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.120024  0.515189 -0.680759 -0.346235 -0.011710  0.160343   
b      2.0  0.650733  0.677469  0.171690  0.411211  0.650733  0.890254   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.332396   3.0  0.411140  1.064341 -0.394612 -0.192136  0.010341   
b     1.129776   2.0  0.055817  0.404322 -0.230081 -0.087132  0.055817   

                          
           75%       max  
key1                      
a     0.814016  1.617691  
b     0.198767  0.341716

### <font color='blue'>**Column-wise and Multiple Function Application**</font>

In [ ]:
tips = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/pandas/tests/io/data/csv/tips.csv")

In [ ]:
tips['tip_pct'] = tips['tip']/tips['total_bill']

In [ ]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


You may want to aggregate using a different function depending on the
column, or multiple functions at once

In [ ]:
grouped = tips.groupby([tips['day'],tips['smoker']])

grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions.

In [ ]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

You don’t need to accept the names that GroupBy gives to the columns; notably,
lambda functions have the name '<lambda>', which makes them hard to identify
(you can see for yourself by looking at a function’s __name__ attribute). Thus, if you
pass a list of (name, function) tuples, the first element of each tuple will be used as
the DataFrame column names (you can think of a list of 2-tuples as an ordered
mapping):

In [ ]:
grouped_pct.agg([('foo','mean'),('bar','std')])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [ ]:
functions = ['count','mean','max']
result = grouped['tip_pct','total_bill'].agg(list(functions))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [ ]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [ ]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [ ]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [ ]:
grouped['tip_pct','total_bill'].agg(ftuples)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [ ]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


Now,suppose you wanted to apply potentially different functions to one or more of the columns. To do this, pass a dict to agg that contains a mapping of column names to any of the function specifications listed so far:

In [ ]:
grouped.agg({'tip': np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [ ]:
grouped.agg({'tip_pct':['mean','min','max','std'],
             'size': 'sum'})

tip_pct                               size
                 mean       min       max       std  sum
day  smoker                                             
Fri  No      0.151650  0.120385  0.187735  0.028123    9
     Yes     0.174783  0.103555  0.263480  0.051293   31
Sat  No      0.158048  0.056797  0.291990  0.039767  115
     Yes     0.147906  0.035638  0.325733  0.061375  104
Sun  No      0.160113  0.059447  0.252672  0.042347  167
     Yes     0.187250  0.065660  0.710345  0.154134   49
Thur No      0.160298  0.072961  0.266312  0.038774  112
     Yes     0.163863  0.090014  0.241255  0.039389   40

### <font color='blue'>**Returning Aggregated Data Without Row Indexes**</font>

In all of the examples up until now, the aggregated data comes back with an index, potentially hierarchical, composed from the unique group key combinations.

Since, this isn't always desirable, you can disable this behavior in more cases by passing `as_index =False to groupby`

In [ ]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# <font color='blue'>**3. Apply: General split-apply-combine**</font>

The most general-purpose `GroupBy` method is `apply`, which is the subject of the rest of the section.
`Apply` splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces together.

First, write a function that selects the rows with largest values by group.

In [ ]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [ ]:
def top(tips,n=5,column='tip_pct'):
  return tips.sort_values(by=column)[-n:]

In [ ]:
top(tips,n=3)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


Now if we group by smoker, and call apply with this function.

In [ ]:
tips.groupby('smoker').apply(top)


total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [ ]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [ ]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]